**Import required libraries and scripts**

In [ ]:
#Import required libraries and scripts
from scripts.library_preparation import *
from scripts.utilities import *
from scripts.docking_functions2 import *
from scripts.clustering_functions import *
from scripts.rescoring_functions import *
from scripts.consensus_methods import *
from scripts.performance_calculation import *
from scripts.dogsitescorer import *
from scripts.get_pocket import *

software = '/home/mario/DockM8/software'
protein_file = '/media/mario/T7/FINISHED/FINISHED/xiap/receptor_protoss_prepared.pdb'
ref_file = '/media/mario/T7/FINISHED/FINISHED/xiap/crystal_ligand_protoss.sdf'
docking_library = '/media/mario/T7/FINISHED/FINISHED/xiap/merged_actives_decoys.sdf'
docking_programs = ['GNINA', 'SMINA', 'PLANTS']
clustering_metrics = ['RMSD', 'spyRMSD', 'espsim', '3DScore', 'bestpose', 'bestpose_GNINA', 'bestpose_SMINA', 'bestpose_PLANTS']
rescoring_functions = ['gnina', 'AD4', 'chemplp', 'rfscorevs', 'LinF9', 'SCORCH', 'RTMScore', 'vinardo']
id_column = 'ID'
n_poses = 10
exhaustiveness = 8
protonation = 'pkasolver'
parallel = 1
ncpus = int(os.cpu_count()/2)
pocket = 'reference'
#Create a temporary folder for all further calculations
w_dir = os.path.dirname(protein_file)
print('The working directory has been set to:', w_dir)
create_temp_folder(w_dir+'/temp')

In [ ]:
if os.path.isfile(protein_file.replace('.pdb', '_pocket.pdb')) == False:
    if pocket == 'reference':
        pocket_definition = GetPocket(ref_file, protein_file, 8)
    elif pocket == 'dogsitescorer':
        pocket_definition = binding_site_coordinates_dogsitescorer(protein_file, w_dir, method='volume')

In [ ]:
if os.path.isfile(w_dir+'/temp/final_library.sdf') == False:
    prepare_library(docking_library, id_column, software, protonation, ncpus)

In [ ]:
docking(w_dir, protein_file, ref_file, software, docking_programs, exhaustiveness, n_poses, ncpus)

In [3]:
print('Loading all poses SDF file...')
tic = time.perf_counter()
all_poses = PandasTools.LoadSDF(w_dir+'/temp/allposes.sdf', idName='Pose ID', molColName='Molecule', includeFingerprints=False, strictParsing=True)
toc = time.perf_counter()
print(f'Finished loading all poses SDF in {toc-tic:0.4f}!...')


Loading all poses SDF file...
Finished loading all poses SDF in 27.7929!...


In [4]:
for metric in clustering_metrics:
        if os.path.isfile(w_dir+f'/temp/clustering/{metric}_clustered.sdf') == False:
            cluster_pebble(metric, 'KMedoids', w_dir, protein_file, all_poses, ncpus)

The folder: /media/mario/T7/FINISHED/FINISHED/xiap/temp/clustering/ was created

[2023-Apr-20 10:26:48]: *Calculating RMSD metrics and clustering*

[2023-Apr-20 10:26:48]: Submitting parallel jobs...


Submitting parallel jobs...: 100%|██████████| 5235/5235 [00:46<00:00, 111.50IDs/s]



[2023-Apr-20 10:27:35]: Finished submitting jobs in 46.9552, now running jobs...


Running clustering jobs...: 100%|██████████| 5235/5235 [35:43<00:00,  2.44jobs/s]  


The folder: /media/mario/T7/FINISHED/FINISHED/xiap/temp/clustering/ already exists

[2023-Apr-20 11:03:23]: *Calculating spyRMSD metrics and clustering*

[2023-Apr-20 11:03:23]: Submitting parallel jobs...


...

The folder: /media/mario/T7/FINISHED/FINISHED/xiap/temp/clustering/ already exists

[2023-Apr-20 11:12:53]: *Calculating 3DScore metrics and clustering*

[2023-Apr-20 11:12:53]: Submitting parallel jobs...


Submitting parallel jobs...: 100%|██████████| 5235/5235 [00:42<00:00, 123.48IDs/s]



[2023-Apr-20 11:13:35]: Finished submitting jobs in 42.3971, now running jobs...


Running clustering jobs...: 100%|██████████| 5235/5235 [07:16<00:00, 11.99jobs/s]  


The folder: /media/mario/T7/FINISHED/FINISHED/xiap/temp/clustering/ already exists

[2023-Apr-20 11:20:53]: *Calculating bestpose metrics and clustering*
The folder: /media/mario/T7/FINISHED/FINISHED/xiap/temp/clustering/ already exists

[2023-Apr-20 11:20:56]: *Calculating bestpose_GNINA metrics and clustering*
The folder: /media/mario/T7/FINISHED/FINISHED/xiap/temp/clustering/ already exists

[2023-Apr-20 11:20:56]: *Calculating bestpose_SMINA metrics and clustering*
The folder: /media/mario/T7/FINISHED/FINISHED/xiap/temp/clustering/ already exists

[2023-Apr-20 11:20:57]: *Calculating bestpose_PLANTS metrics and clustering*


In [5]:
for metric in clustering_metrics:
        rescore_all(w_dir, protein_file, ref_file, software, w_dir+f'/temp/clustering/{metric}_clustered.sdf', rescoring_functions, ncpus)


The folder: /media/mario/T7/FINISHED/FINISHED/xiap/temp/rescoring_RMSD_clustered was created
Splitting SDF file RMSD_clustered.sdf ...


Splitting files: 100%|██████████| 25/25 [00:02<00:00, 11.95it/s]


Split docking library into 25 files each containing 667 compounds

[2023-Apr-20 11:21:06]: Rescoring with GNINA


Rescoring with GNINA: 100%|██████████| 25/25 [02:41<00:00,  6.47s/file]



[2023-Apr-20 11:23:50]: Rescoring with GNINA complete in 170.5453!

[2023-Apr-20 11:23:50]: Rescoring with AD4

[2023-Apr-20 11:24:11]: Rescoring with AD4 complete in 20.1933!

[2023-Apr-20 11:24:11]: Rescoring with CHEMPLP


...


[2023-Apr-20 12:56:46]: Rescoring with LinF9 complete in 4.9737!
The folder: /media/mario/T7/FINISHED/FINISHED/xiap/temp/rescoring_bestpose_PLANTS_clustered/SCORCH_rescoring/ was created

[2023-Apr-20 12:56:46]: Converting protein file to .pdbqt ...

[2023-Apr-20 12:56:48]: Converting SDF file bestpose_PLANTS_clustered.sdf to .pdbqt files...
Converted 5003 molecules.

[2023-Apr-20 12:56:55]: Rescoring with SCORCH

[2023-Apr-20 12:58:26]: Rescoring with SCORCH complete in 99.2261!
The folder: /media/mario/T7/FINISHED/FINISHED/xiap/temp/rescoring_bestpose_PLANTS_clustered/RTMScore_rescoring/ was created

[2023-Apr-20 12:58:26]: Rescoring with RTMScore

[2023-Apr-20 13:01:13]: Rescoring with RTMScore complete in 167.4117!


'/media/mario/T7/FINISHED/FINISHED/xiap/temp/clustering/bestpose_PLANTS_clustered.sdf'


[2023-Apr-20 13:01:13]: Rescoring with Vinardo

[2023-Apr-20 13:01:19]: Rescoring with Vinardo complete in 5.6375!

[2023-Apr-20 13:01:19]: Combining all score for /media/mario/T7/FINISHED/FINISHED/xiap/temp/rescoring_bestpose_PLANTS_clustered


Combining scores: 100%|██████████| 7/7 [00:00<00:00, 713.89files/s]


[2023-Apr-20 13:01:19]: Rescoring complete in 391.3670!


In [6]:
calculate_EF_single_functions(w_dir, docking_library, clustering_metrics)
apply_consensus_methods_combinations(w_dir, docking_library, clustering_metrics)

The folder: /media/mario/T7/FINISHED/FINISHED/xiap/temp/ranking was created
The folder: /media/mario/T7/FINISHED/FINISHED/xiap/temp/consensus was created
The folder: /media/mario/T7/FINISHED/FINISHED/xiap/temp/ranking already exists
The folder: /media/mario/T7/FINISHED/FINISHED/xiap/temp/consensus already exists

[2023-Apr-20 15:13:34]: Calculating consensus methods for every possible score combination...


100%|██████████| 8/8 [03:55<00:00, 29.47s/it]


In [ ]:
def apply_consensus_methods(w_dir, clustering_metric, method, rescoring_functions):
    create_temp_folder(w_dir+'/temp/ranking')
    rescoring_folder = f'rescoring_{clustering_metric}_clustered'
    rescored_dataframe = pd.read_csv(w_dir + f'/temp/{rescoring_folder}/allposes_rescored.csv')
    standardised_dataframe = standardize_scores(rescored_dataframe)
    col_dict = {'gnina':['GNINA', 'CNN-Score', 'CNN-Affinity'], 'vinardo':'Vinardo', 'AD4':'AD4', 'LinF9':'LinF9', 'rfscorevs':'RFScoreVS', 'plp':'PLP', 'chemplp':'CHEMPLP', 'NNScore':'NNScore', 
               'PLECnn':'PLECnn', 'AAScore':'AAScore', 'ECIF':'ECIF', 'SCORCH':'SCORCH','RTMScore':'RTMScore'}
    col_list = ['Pose ID', 'GNINA', 'CNN-Score', 'RTMScore']
    # for function in rescoring_functions:
    #     cols = col_dict[function]
    #     if isinstance(cols, list):
    #         col_list.extend(cols)
    #     else:
    #         col_list.append(cols)
    # print(col_list)
    filtered_dataframe = standardised_dataframe[col_list]
    print(filtered_dataframe)
    standardised_dataframes, ranked_dataframes = process_dataframes(w_dir, {clustering_metric: rescoring_folder})
    for name, df_dict in {'standardised': standardised_dataframes, 'ranked': ranked_dataframes}.items():
        for df_name, df in df_dict.items():
            df['ID'] = df['Pose ID'].str.split('_').str[0]
            df.to_csv(w_dir + f'/temp/ranking/{df_name}.csv', index=False)

    create_temp_folder(w_dir+'/temp/consensus')
    rank_methods = {'method1': method1_ECR_best, 'method2': method2_ECR_average, 'method3': method3_avg_ECR, 'method4': method4_RbR}
    score_methods = {'method5': method5_RbV, 'method6': method6_Zscore_best, 'method7': method7_Zscore_avg}

    if method in rank_methods:
        method_function = rank_methods[method]
        analysed_dataframe = method_function(ranked_dataframes[clustering_metric+'_ranked'], clustering_metric, [col for col in ranked_dataframes[clustering_metric+'_ranked'] if col not in ['Pose ID', 'ID']])
    elif method in score_methods:
        method_function = score_methods[method]
        analysed_dataframe = method_function(standardised_dataframes[clustering_metric+'_standardised'], clustering_metric, [col for col in standardised_dataframes[clustering_metric+'_standardised'] if col not in ['Pose ID', 'ID']])
    else:
        raise ValueError(f"Invalid method: {method}")

    print(analysed_dataframe)
    analysed_dataframe = analysed_dataframe.drop(columns="Pose ID", errors='ignore')
    analysed_dataframe.to_csv(w_dir+f'/temp/consensus/{clustering_metric}_{method}_results.csv', index=False)

apply_consensus_methods(w_dir, 'bestpose', 'method6', ['gnina', 'RTMScore'])

In [ ]:
results = pd.read_csv('/home/mario/DockM8/cdk2/temp/consensus/bestpose_method6_results.csv')
database = PandasTools.LoadSDF('/home/mario/DockM8/cdk2/temp/final_library.sdf', molColName=None, smilesName='SMILES', idName='ID')
final = pd.merge(database, results, on='ID', how='inner')
final.to_csv('/home/mario/DockM8/cdk2/temp/consensus/results_for_ML.csv', index=False)

In [ ]:
./gnina -r /media/mario/T7/FINISHED/FINISHED/ace/receptor_prepared_protoss.pdb -l /media/mario/T7/FINISHED/FINISHED/ace/temp/clustering/RMSD_clustered.sdf --autobox_ligand /media/mario/T7/FINISHED/FINISHED/ace/crystal_ligand_protoss.sdf -o /media/mario/T7/FINISHED/FINISHED/ace/temp/rescoring_RMSD_clustered/vinardo_rescoring/vinardo_scores.csv --score_only --scoring vinardo --cnn_scoring none --no_gpu